In [1]:
import numpy as np

### Broadcast

**The Broadcasting Rule**

>In order to broadcast, the size of the trailing axes for both arrays in an operation must either be the same size or one of them must be one.


``` ndarray(n,m,l) * ndarray(m,l) ``` OR ``` ndarray(n,m,l) * ndarray(1,1) ```

In [6]:
%%prun
a * 8

         3 function calls in 0.452 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.452    0.452    0.452    0.452 <string>:1(<module>)
        1    0.000    0.000    0.452    0.452 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [10]:
%%prun
eight = np.full(a.shape,8)
a * eight

         10 function calls in 3.722 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    3.492    3.492    3.663    3.663 <string>:1(<module>)
        1    0.171    0.171    0.171    0.171 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.059    0.059    3.722    3.722 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.empty}
        1    0.000    0.000    0.171    0.171 numeric.py:288(full)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.array}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.171    0.171 <__array_function__ internals>:2(copyto)
        1    0.000    0.000    0.000    0.000 _asarray.py:23(asarray)
        1    0.000    0.000    0.000    0.000 multiarray.py:1054(copyto)

In [7]:
x = np.arange(5)
x[:,np.newaxis] + x[np.newaxis,:]

array([[0, 1, 2, 3, 4],
       [1, 2, 3, 4, 5],
       [2, 3, 4, 5, 6],
       [3, 4, 5, 6, 7],
       [4, 5, 6, 7, 8]])

#### Vector quantization

![VQ](https://numpy.org/doc/stable/_images/theory.broadcast_5.png)

In [18]:
bp = np.array([[205],[112]])
fl = np.array([[195],[148]])
mr = np.array([[175],[68]])
fg = np.array([[155],[50]])

In [19]:
new = np.array([[187],[126]])

In [20]:
athletes = np.hstack((bp,fl,mr,fg))
athletes

array([[205, 195, 175, 155],
       [112, 148,  68,  50]])

**Euclidean distance**
$$
\sqrt{\sum_{i = 1}^{n}(p_i - q_i)^2} 
$$
**Finding closest** $ {Athletes}_c\ , c = \arg \min \sqrt{\sum_{i = 1}^{n}(p_i - q_i)^2}  $

In [23]:
arg = np.argmin(
    np.sqrt(
        np.sum(
            np.power( (athletes - new), 2)
            ,axis=0)
    ) 
)

In [34]:
athletes[:,arg] ##backetball player

array([205, 112])

### Structured arrays

In [75]:
struct = np.array([('Bob',42), ('Alice',39)],
                  dtype = [('name','U5'),('age','i1')])

In [76]:
struct

array([('Bob', 42), ('Alice', 39)], dtype=[('name', '<U5'), ('age', 'i1')])

In [77]:
struct['name']

array(['Bob', 'Alice'], dtype='<U5')

### Indexing

In [2]:
a = np.arange(12).reshape(3,4)
i = np.array([[1,2],[0,1]])
j = i.view()

In [3]:
a[i]

array([[[ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[ 0,  1,  2,  3],
        [ 4,  5,  6,  7]]])

In [4]:
a[i,j]

array([[ 5, 10],
       [ 0,  5]])

In [5]:
a[i][0]

array([[ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [6]:
a[i].shape

(2, 2, 4)

#### Note:
`a[i,j] is faster than a[i][j]`

In [29]:
%%prun
a = np.random.rand(40,12,15,12,9,6,10)

         4 function calls in 0.771 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.598    0.598    0.598    0.598 {method 'rand' of 'numpy.random.mtrand.RandomState' objects}
        1    0.154    0.154    0.752    0.752 <string>:1(<module>)
        1    0.019    0.019    0.771    0.771 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [30]:
%%time
a[2,7,3,1,4,5,0]

Wall time: 69.2 ms


0.9492695360693872

In [31]:
%%time
a[2][7][3][1][4][5][0]

Wall time: 0 ns


0.9492695360693872

In [16]:
a[2,7,3,1,4,Ellipsis,0]

array([0.56519612, 0.982017  , 0.25683676, 0.23245569, 0.19628817,
       0.77130088])

### Creation and manipulation

In [5]:
a = np.zeros([5,3])
a

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [6]:
b = np.identity(5)
b

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [7]:
%%time
b.dot(a)

Wall time: 144 ms


array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [8]:
c = np.full([4,3], 7); c

array([[7, 7, 7],
       [7, 7, 7],
       [7, 7, 7],
       [7, 7, 7]])

In [11]:
b.dot(a).dot(c.T).reshape(2,-1)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [12]:
c.reshape(1,4*3)

array([[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]])

In [9]:
m = np.matrix([
    [1,2,3],
    [7,5,9],
    [12,-1,0]
])

In [10]:
np.rollaxis(m,1)

matrix([[ 1,  7, 12],
        [ 2,  5, -1],
        [ 3,  9,  0]])

In [11]:
np.concatenate((m,c.reshape(3,-1)),axis=1)

matrix([[ 1,  2,  3,  7,  7,  7,  7],
        [ 7,  5,  9,  7,  7,  7,  7],
        [12, -1,  0,  7,  7,  7,  7]])

In [12]:
np.split(np.concatenate((m,c.reshape(3,-1)),axis=1),7,axis=1)

[matrix([[ 1],
         [ 7],
         [12]]),
 matrix([[ 2],
         [ 5],
         [-1]]),
 matrix([[3],
         [9],
         [0]]),
 matrix([[7],
         [7],
         [7]]),
 matrix([[7],
         [7],
         [7]]),
 matrix([[7],
         [7],
         [7]]),
 matrix([[7],
         [7],
         [7]])]

In [13]:
np.unique(c,return_counts=True)

(array([7]), array([12], dtype=int64))

In [14]:
np.flip(m,axis=[1,0])

matrix([[ 0, -1, 12],
        [ 9,  5,  7],
        [ 3,  2,  1]])

In [15]:
np.ravel(m)

array([ 1,  2,  3,  7,  5,  9, 12, -1,  0])

In [16]:
np.insert(m,obj=[0,1,2],values=[-7,-7,-7],axis=1)

matrix([[-7,  1, -7,  2, -7,  3],
        [-7,  7, -7,  5, -7,  9],
        [-7, 12, -7, -1, -7,  0]])

### Strings and datetime

In [21]:
np.char.count('aff0vs','f')

array(2)

In [22]:
np.char.find('abc','b')

array(1)